# Анализ базы данных сервиса для чтения книг по подписке

Вы работаете в компании, которая компания купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Цель проекта - проанализировать данные, чтобы помочь в дальнейшем сформулировать новое предложение для клиентов.

Задачи:
- выгрузить информацию из базы данных;
- исследовать полученные таблицы;
- проанализировать данные и ответить на поставленные вопросы.

## Изучение и предобработка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# функция для чтения файлов
def select_sql(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

Выгрузим таблицы из базы данных.

Таблица books содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [4]:
books = '''
SELECT
    *
FROM 
    books
'''
select_sql(books).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
books_cnt = '''
SELECT
    COUNT(*) AS cnt
FROM 
    books
'''
select_sql(books_cnt)

,cnt
0,1000


Таблица books состоит из 1000 строк и 6 столбцов.

Таблица authors cодержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

In [6]:
authors = '''
SELECT 
    *
FROM 
    authors
'''
select_sql(authors).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
authors_cnt = '''
SELECT
    COUNT(*) AS cnt
FROM 
    authors
'''
select_sql(authors_cnt)

,cnt
0,636


Таблица authors состоит из 636 строк и 2 столбца.

Таблица publishers cодержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

In [8]:
publishers = '''
SELECT 
    *
FROM 
    publishers
'''
select_sql(publishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
publishers_cnt = '''
SELECT
    COUNT(*) AS cnt
FROM 
    publishers
'''
select_sql(publishers_cnt)

,cnt
0,340


Таблица publishers состоит из 340 строк и 2 столбца.

Таблица ratings cодержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [10]:
ratings = '''
SELECT 
    *
FROM 
    ratings
'''
select_sql(ratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
ratings_cnt = '''
SELECT
    COUNT(*) AS cnt
FROM 
    ratings
'''
select_sql(ratings_cnt)

,cnt
0,6456


Таблица ratings состоит из 6456 строк и 4 столбцов.

Таблица reviews cодержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- rating — текст обзора.

In [12]:
reviews = '''
SELECT 
    *
FROM 
    reviews
'''
select_sql(reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
reviews_cnt = '''
SELECT
    COUNT(*) AS cnt
FROM 
    reviews
'''
select_sql(reviews_cnt)

,cnt
0,2793


Таблица reviews состоит из 2793 строк и 4 столбцов.

### Вывод

На данном этапе:
- мы выгрузили таблицы из базы данных;
- исследовали полученные таблицы.

## Анализ данных

Посчитаем количество книг, вышедших после 1 января 2000 года.

In [14]:
books_qty = '''
SELECT
    COUNT(book_id)
FROM 
    books
WHERE 
    publication_date >= '2000-01-01'
'''
select_sql(books_qty)

,count
0,821


Начиная с 1 января 2000 года, вышло 821 книга.

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [15]:
reviews_qty_and_ratings = '''
SELECT
    books.book_id AS book_id,
    books.title AS title,
    COUNT(DISTINCT reviews.text) AS cnt_reviews,
    AVG(ratings.rating) AS avg_rating
FROM
    books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
     books.title,
     books.book_id
ORDER BY
     cnt_reviews DESC
'''
select_sql(reviews_qty_and_ratings)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Для каждой книги посчитаем количество обзоров и среднюю оценку.
На первом месте по количеству обзоров книга "Twilight (Twilight #1)", но при этом рейтинг у неё достаточно низкий - 3.66.
Несколько книг без обзоров, но с рейтингом.

In [16]:
select_sql(ratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [17]:
select_sql(reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
select_sql(publishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [19]:
books_qty_per_publisher = '''
SELECT
    publishers.publisher_id AS publisher_id,
    publishers.publisher AS publisher,
    COUNT(DISTINCT books.book_id) AS cnt_books
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
     publishers.publisher_id
ORDER BY
     cnt_books DESC
'''
select_sql(books_qty_per_publisher)

,publisher_id,publisher,cnt_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19
...,...,...,...
329,238,Random House Anchor,1
330,239,Random House Audio Publishing Group,1
331,241,Random House Large Print Publishing,1
332,242,Random House Trade,1


Издательство "Penguin Books" выпустило больше всего книг - 42.

Определим автора с самой высокой средней оценкой книг, количество оценок 50 и более.

In [20]:
authors_and_avg_ratings = '''
SELECT
    SUBQ.author_id AS author_id,
    SUBQ.author AS author,
    AVG(SUBQ.avg_rating) AS avg_rating
FROM
    (SELECT
        authors.author_id AS author_id,
        authors.author AS author,
        books.book_id AS book_id,
        books.title AS title,
        AVG(ratings.rating) AS avg_rating,
        COUNT(ratings.rating) AS cnt_rating
    FROM
        authors
    INNER JOIN books ON books.author_id = authors.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        authors.author_id,
        books.book_id
    HAVING
        COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author_id,
    author
ORDER BY
     avg_rating DESC
'''
select_sql(authors_and_avg_ratings)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,106,Dan Brown,3.754540


J.K. Rowling/Mary GrandPré - авторы с самой высокой средней оценкой книг и количеством оценок 50 и более. Средняя оценка книг этих авторов - 4.28 баллов.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [21]:
avg_reviews = '''
SELECT
    AVG(SUBQ.cnt_reviews) AS avg_reviews
FROM
    (SELECT
        reviews.username AS username,
        COUNT(reviews.review_id) AS cnt_reviews
    FROM
        reviews
    WHERE
        username IN
            (SELECT
                ratings.username AS username
            FROM
                ratings
            GROUP BY
                username
            HAVING
                COUNT(ratings.rating_id) > 50)
    GROUP BY
        username) AS SUBQ  
'''
select_sql(avg_reviews)

,avg_reviews
0,24.333333


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, составило 24 обзора.

### Вывод

На данном этапе:
- мы проанализировали полученные таблицы;
- количество книг, вышедших с 1 января 2000 года, составило 821;
- для каждой книги посчитали количество обзоров и их среднюю оценку. На первом месте по количеству обзоров книга "Twilight (Twilight #1)" - 7 обзоров, а её рейтинг невысокий - 3.66;
- на первом месте по количеству выпущенных книг находится издательство "Penguin Books" (42 книги);
- на первом месте среди авторов с самой высокой средней оценкой книг и количеством оценок 50 и более находятся J.K. Rowling/Mary GrandPré. Средняя оценка их книг - 4.28 баллов;
- среднее количество обзоров от пользователей, поставивших больше 50 оценок, равно 24 обзорам.

## Общий вывод

В ходе данного исследования были сделаны следующие шаги:
- извлечены таблицы из базы данных;
- проанализированы полученные данные;
- были получены ответы на поставленные вопросы.